## cvxpy
В этой практике предполагается использование пакета [`cvxpy`](https://www.cvxpy.org/). Этот пакет представляет собой DSL для описания выпуклых задач оптимизации и поставляется вместе с open-source бэкендом для решения задач, при этом его можно совместить и с другими оптимизационными [бэкэндами](https://www.cvxpy.org/install/index.html#install-with-cvxopt-and-glpk-support). `cvxpy` довольно прост в использовании, но стоит учитывать, что с помощью него можно описывать только выпуклые задачи оптимизации, выпуклость строго отслеживается при составлении задачи. Работа с `cvxpy` состоит из 4ех примитивов:

### Переменные

In [1]:
import numpy as np
import cvxpy as cp

m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)
x = cp.Variable(n) 

### Целевая функция

In [2]:
objective = cp.Minimize(cp.sum_squares(A @ x - b))

### Ограничения

In [3]:
constraints = [0 <= x, x <= 1]

### Задача оптимизации

In [4]:
prob = cp.Problem(objective, constraints)
prob.solve() # возвращает оптимальное значение

19.83126370644502

После вызова метода `solve()` из переменных можно извлечь оптимальные значения с помощью атрибута `value`, в ограниченях атрибут `dual_value` содержит оптимальные значения переменных двойственной задачи.

In [5]:
print(x.value)
print(constraints[0].dual_value)

[-1.79109255e-19  2.85112420e-02  2.79973443e-19  3.37658729e-20
 -2.72802663e-19  1.49285011e-01 -9.94082533e-20  8.35373900e-20
  2.46718649e-01  5.78224144e-01 -4.03739463e-19  1.01242860e-03
 -9.28486180e-20  2.26767464e-01 -1.58813678e-19 -8.97232272e-20
 -1.22145729e-19 -1.51509428e-19  1.12060672e-19 -3.48318635e-19]
[ 2.50938945  0.          2.78354615  1.79425782 13.08579183  0.
  0.73716363  3.35344995  0.          0.          8.93825054  0.
  7.02955161  0.          4.71068649  3.18873635  2.06090107 10.08166738
  3.0481157   8.53268239]


# Задача #1: Задача о кратчайших путях
Используйте `cvxpy`, чтобы найти дерево кратчайших путей из вершины $s$ во все остальные вершины.

In [ ]:
from typing import List, Tuple, Dict, Set
def shortest_path_tree(s: int,
                       arcs: List[Tuple[int, int, float]]) -> Tuple[Dict[int, float], Dict[int, int]]:
    """
    Строит дерево кратчайших путей для орграфа: для каждой вершины не считая s
    найти последнее ребро на одного из минимальныъ путей из s в эту вершину
    
    Args:
        s: вершина, из которой искать кратчайшие пути
        arcs: список ребер в формате (вершина-начало, вершина-конец, длина)
        
    Returns:
        d: для каждой вершины расстояние от s до нее
        prev: для каждой вершины v кроме s номер ребра в arcs, которое является последнем в 
        одном из минимальных путей из s в v 
    """
pass

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from graph_utils.graph import Graph, Arc

arcs = [
    (0, 1, 4),
    (0, 2, 2),
    (1, 2, 5),
    (2, 3, 3),
    (1, 4, 10),
    (3, 4, 4),
    (4, 5, 11)
]
s = 0

d, prev = shortest_path_tree(s, arcs)
graph = Graph([Arc(arc[0], arc[1], arc[2], attributes={"color": "red" if prev[arc[1]] == i else "black"}) for i, arc in enumerate(arcs)])
graph.Visualize({key: f"{x:.2f}" for key, x in d.items()})

In [ ]:
arcs = [
    (1, 3, 3),
    (1, 4, 7),
    (4, 3, 2),
    (4, 5, 3),
    (1, 5, 2),
    (6, 4, 2),
    (5, 6, 2),
    (6, 7, 1),
    (7, 2, 7),
    (4, 2, 2),
    (3, 2, 5)
]
s = 1
d, prev = shortest_path_tree(s, arcs)
print({key: f"{x:.2f}" for key, x in d.items()}, prev)
graph = Graph([Arc(arc[0], arc[1], arc[2], attributes={"color": "red" if prev[arc[1]] == i else "black"}) for i, arc in enumerate(arcs)])
graph.Visualize({key: f"{x:.2f}" for key, x in d.items()})

Ожидаемый вывод либо
```
{1: '0.00', 2: '8.00', 3: '3.00', 4: '6.00', 5: '2.00', 6: '4.00', 7: '5.00'} {3: 0, 5: 4, 4: 5, 6: 6, 7: 7, 2: 10}
```
либо
```
{1: '0.00', 2: '8.00', 3: '3.00', 4: '6.00', 5: '2.00', 6: '4.00', 7: '5.00'} {3: 0, 5: 4, 4: 5, 6: 6, 7: 7, 2: 9}
```

## Задача #2: максимальный поток, минимальный s-t разрез
Дан орграф $G=\langle V, E\rangle$ с двумя выделенными вершинами $s, t$ и пропускными способностями $c$. Требуется найти такое разбиение вершин графа $S\cup T=V,~S\cap T=\emptyset$, $s\in S, t\in T$ такое, что величина
$$
\sum_{u\in S, v\in T}c_{uv}
$$
минимальна. Эта задача является двойственной к задаче о максимальном потоке в том числе и в смысле двойственности Лагранжа.

<details>
<summary>Подсказка</summary>
    задача о максимальном потоке может быть представлена в виде задачи линейного программирования например следуюим образом
$$
\begin{array}{ll}
\mbox{максимизировать } & f\\
\mbox{при условии }    & Bx=f\chi^{st}\\
      & 0\leq x\leq c.
\end{array}\tag{4}
$$
</details>

In [ ]:
def st_cut(s: int,
           t: int,
           arcs: List[Tuple[int, int, float]]) -> Tuple[List[int], List[int], Set[int]]:
    """
    Строит дерево кратчайших путей для орграфа: для каждой вершины не считая s
    найти последнее ребро на одного из минимальныъ путей из s в эту вершину
    
    Args:
        s: вершина, которую следует отделить от t
        t: вершина, которую следует отделить от s
        arcs: список ребер в формате (вершина-начало, вершина-конец, длина)
        
    Returns:
        S: список вершин в минимальном разрезе, содержащий s
        T: список вершин в минимальном разрезе, содержащий t
        cut: множество индексов ребер, разделяющих S и T
    """
    pass

In [ ]:
s, t = 1, 2
S, T, cut = st_cut(s, t, arcs)
print(S, T)
graph = Graph([Arc(arc[0], arc[1], arc[2],
                    attributes={"color": "red" if i in cut else "black", "constraint": str(i in cut)}) for i, arc in enumerate(arcs)])
graph.nodes[s].SetColor('green')
graph.nodes[t].SetColor('blue')
        
graph.Visualize()

Ожидаемый вывод либо
```
[1, 4, 5, 6] [2, 3, 7]
```
либо
```
[1, 3, 4, 5, 6] [2, 7]
```